In [ ]:
import re

In [ ]:
from pathlib import Path

import iris
import matplotlib.pyplot as plt
from wildfires.data import homogenise_time_coordinate

In [ ]:
data_dir = Path("/work/scratch-nopw/alexkr/fluxnet/run11a/jules_output")
cube_files = sorted(list(data_dir.glob("*.D.*.nc")))
cube_files

In [ ]:
def add_location_callback(cube, field, filename):
    """Add the location from the filename as a Cube attribute."""
    cube.attributes["location"] = re.match(
        "^(.*)-JULES", str(Path(filename).name)
    ).group(1)

In [ ]:
raw_cubes = iris.load(map(str, cube_files), callback=add_location_callback)

In [ ]:
raw_cubes = homogenise_time_coordinate(raw_cubes).concatenate()

#### Get variable names

In [ ]:
variable_names = sorted({cube.name() for cube in raw_cubes})
len(variable_names), variable_names

In [ ]:
locations = sorted({cube.attributes["location"] for cube in raw_cubes})
len(locations), locations

In [ ]:
# raw_cubes.extract(iris.AttributeConstraint(location=raw_cubes[0].attributes['location']))

In [ ]:
cube.shape

In [ ]:
variables = [
    "Gridbox net primary production prior to N limitation",
    "Gridbox gross primary productivity",
    "PFT leaf area index",
]

for location in locations:
    scubes = raw_cubes.extract(iris.AttributeConstraint(location=location))
    fig, ax = plt.subplots(figsize=(7, 3), dpi=180)
    fig.suptitle(location)
    lines = []
    for var, c in zip(variables, ["C0", "C1", "C2"]):
        ecubes = scubes.extract(iris.Constraint(name=var))
        if not ecubes:
            continue

        cube = ecubes[0]
        if cube.shape[1] == 9:
            ax2 = ax.twinx()
            # Loop over PFTs
            for i in range(9):
                line = ax2.plot(
                    cube.data[:, i].ravel(), label=var, c=c, ls="--", alpha=0.8
                )[0]
                if not i:
                    lines.append(line)
        else:
            lines.append(ax.plot(cube.data.ravel(), label=var, c=c)[0])
    ax.legend(
        lines,
        [l.get_label() for l in lines],
        loc="lower left",
        bbox_to_anchor=(0.58, 1),
    )